## What this notebook is
This notebook demonstrates how I trained Gemma-2 9b to obtain LB: 0.941. The inference code can be found [here](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora).
I used 4-bit quantized [Gemma 2 9b Instruct](https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit) uploaded by unsloth team as a base-model and added LoRA adapters and trained for 1 epoch.

## Result

I used `id % 5 == 0` as an evaluation set and used all the rest for training.

| subset | log loss |
| - | - |
| eval | 0.9371|
| LB | 0.941 |

## What is QLoRA fine-tuning?

In the conventional fine-tuning, weight ($\mathbf{W}$) is updated as follows:

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

where $L$ is a loss at this step and $\eta$ is a learning rate.

[LoRA](https://arxiv.org/abs/2106.09685) tries to approximate the $\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$ by factorizing $\Delta \mathbf{W}$ into two (much) smaller matrices, $\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$ and $\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$ with $r \ll \text{min}(\text{d}, \text{k})$.

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

During training, only $\mathbf{A}$ and $\mathbf{B}$ are updated while freezing the original weights, meaning that only a fraction (e.g. <1%) of the original weights need to be updated during training. This way, we can reduce the GPU memory usage significantly during training while achieving equivalent performance to the usual (full) fine-tuning.

[QLoRA](https://arxiv.org/abs/2305.14314) pushes the efficiency further by quantizing LLM. For example, a 8B parameter model alone would take up 32GB of VRAM in 32-bit, whereas quantized 8-bit/4-bit 8B model only need 8GB/4GB respectively. 
Note that QLoRA only quantize LLM's weights in low precision (e.g. 8-bit) while the computation of forward/backward are done in higher precision (e.g. 16-bit) and LoRA adapter's weights are also kept in higher precision.

1 epoch using A6000 took ~15h in 4-bit while 8-bit took ~24h and the difference in log loss was not significant.

## Note
It takes prohivitively long time to run full training on kaggle kernel. I recommend to use external compute resource to run the full training.
This notebook uses only 100 samples for demo purpose, but everything else is same as my setup.

In [1]:
# # gemma-2 is available from transformers>=4.42.3
# !pip install -U "transformers==4.51" bitsandbytes accelerate peft

In [2]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Qwen2ForSequenceClassification,
    Qwen2TokenizerFast,
    Qwen2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

### Configurations

In [3]:
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "../Qwen2.5-7B-Instruct"  # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 2048
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 2  # global batch size is 8 
    per_device_eval_batch_size: int = 8
    n_epochs: int = 5
    freeze_layers: int = 0  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()

#### Training Arguments

In [4]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=200,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

#### LoRA config

In [5]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

### Instantiate the tokenizer & model

In [6]:
tokenizer = Qwen2TokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token  # 很多中文模型没 pad_token，可复用 eos

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # or 'fp4'
    bnb_4bit_compute_dtype=torch.float16,
)

model = Qwen2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=2,
    quantization_config=bnb_config,
    # torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id
# model

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at ../Qwen2.5-7B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# for name, param in model.named_parameters():
#     if not param.dtype.is_floating_point:
#         print(f"Non-float parameter: {name}, dtype: {param.dtype}")


In [9]:
model.print_trainable_parameters()

trainable params: 10,099,712 || all params: 7,080,726,016 || trainable%: 0.1426


### Instantiate the dataset

In [10]:
ds = Dataset.from_csv("../data/all_insurance_data_for_classification.csv")
# ds = ds.select(torch.arange(800))  # We only use the first 100 data for demo purpose

In [11]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        prompt = [f"<prompt>: 下面保险文本内容是否与{t}相关?" for t in batch["module"]]
        insurance_text = ["\n\n保险文本: " + t for t in batch["text_split"]]
        texts = [p + r_a for p, r_a in zip(prompt, insurance_text)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels=[int(t) for t in batch["neg_or_pos"]]
        
        return {**tokenized, "labels": labels}
        

In [12]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

Map:   0%|          | 0/1910 [00:00<?, ? examples/s]

### Compute metrics

We'll compute the log-loss used in LB and accuracy as a auxiliary metric.

In [13]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

### Split

Here, train and eval is splitted according to their `id % 5`

In [14]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]

In [15]:
# ds.select(train_idx)

In [ ]:
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

/tmp/ipykernel_3350/2739703425.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Acc,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,0.913000,0.934976,0.541885,0.934977,11.412100,33.473000,4.206000
2,0.491800,0.759979,0.753927,0.759989,11.421400,33.446000,4.203000
3,0.476200,0.601521,0.840314,0.601537,11.447600,33.369000,4.193000
4,0.170200,0.472220,0.887435,0.472204,11.448600,33.367000,4.193000


/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/torch/utils/checkpoint.p